In [2]:
import torch
import torch.nn as nn
import math

# LayerNorm



## 核心思想

对每个样本（token）自己的所有特征独立做“均值为 0、标准差为 1”的归一化，完全不依赖 batch 大小。

![LN](pic/LN.png)

## 为什么这么做

由于对每个样本做了归一化，首先带来的好处是梯度是稳定的，不会过大或者过小，而梯度稳定则是训练稳定的重要前提。

In [3]:

class layernorm(nn.Module):
    def __init__(self, input_dim, eps=1e-5) -> None:
        super().__init__()
        self.eps = eps
        self.input_dim = input_dim
        self.weight = nn.Parameter(torch.ones(self.input_dim))
        self.bias = nn.Parameter(torch.zeros(self.input_dim))

    def forward(self, x):
        mean = x.mean(dim= -1, keepdim=True)
        var = x.var(dim= -1, keepdim=True, unbiased=False)
        x_norm = (x - mean) / torch.sqrt(var + self.eps)
        return self.weight * x_norm + self.bias

## 代码测试

In [4]:
x = torch.tensor([[1., 2., 3., 4.], [10., 20., 30., 40.]], dtype=torch.float32)
ln = layernorm(4)
x_norm = ln(x)
x_norm

tensor([[-1.3416, -0.4472,  0.4472,  1.3416],
        [-1.3416, -0.4472,  0.4472,  1.3416]], grad_fn=<AddBackward0>)

## 一些值得注意的

### `nn.Parameter()`的用法

* `nn.Parameter()`用于自定义可学习的参数，是tensor的子类。

    当神经网络继承自nn.Module时，如果其中存在一些参数是需要纳入模型本身的参数里参与反向传播过程的，则可以使用该方法创建参数。

* 创建方法即为使用`nn.Parameter()`方法包裹tensor:

    `self.weight = nn.Parameter(torch.ones(self.input_dim))`


# RMS Norm



## 核心思想

![LN](pic/RMS_Norm.png)

RMSNorm 主要是在 LayerNorm 的基础上去掉了减均值这一项，其计算效率更高且没有降低性能。

因为原论文中指出LayerNorm发挥作用主要是其缩放不变性（即除以标准差），而不是平移不变性（即减去均值）


In [5]:
class RMS_Norm(nn.Module):
    def __init__(self, input_dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.input_dim = input_dim
        self.weight = nn.Parameter(torch.ones(self.input_dim))
    def forward(self, x):
        rms = torch.sqrt(x.pow(2).mean(dim=-1, keepdim=True) + self.eps)
        return x * self.weight / rms


## 代码测试

In [6]:
x = torch.tensor([[1., 2., 3., 4.], [10., 20., 30., 40.]], dtype=torch.float32)
rmsn = RMS_Norm(4)
x_norm = ln(x)
x_norm

tensor([[-1.3416, -0.4472,  0.4472,  1.3416],
        [-1.3416, -0.4472,  0.4472,  1.3416]], grad_fn=<AddBackward0>)

# Batch Norm

# MLP


In [7]:
class MLP(nn.Module):
    def __init__(self, input_dim) -> None:
        super().__init__()
        self.input_dim = input_dim
        self.w1 = nn.Linear(input_dim, input_dim * 4)
        self.relu = nn.ReLU()
        self.w2 = nn.Linear(input_dim * 4, input_dim)
    def forward(self, x):
        return self.w2(self.relu(self.w1(x)))


## 代码测试

In [8]:
x = torch.rand(2, 3, 24)
mlp = MLP(24)
x = mlp(x)
print("x.shape:", x.shape)
x

x.shape: torch.Size([2, 3, 24])


tensor([[[ 0.2339, -0.0391, -0.2736,  0.3050,  0.1875,  0.2040,  0.1997,
          -0.0726, -0.2280,  0.0865, -0.1783, -0.0506,  0.1255, -0.1971,
          -0.1416,  0.1578,  0.1998, -0.0239, -0.2228,  0.0019, -0.2107,
          -0.2518, -0.0898,  0.2227],
         [ 0.3744, -0.2178, -0.2694,  0.1846,  0.1348,  0.2065,  0.2043,
          -0.0200, -0.1848,  0.2657, -0.2589, -0.0554,  0.0434, -0.1839,
          -0.2362, -0.0133,  0.0999, -0.0063, -0.2062,  0.0995, -0.1481,
          -0.2620, -0.3212, -0.0294],
         [ 0.3378, -0.1335, -0.1618,  0.1279,  0.2274,  0.1612,  0.1971,
          -0.0350, -0.0987,  0.0697, -0.2664,  0.0291,  0.0427, -0.1909,
          -0.1831,  0.1385,  0.0588, -0.0115, -0.3154,  0.0756, -0.1671,
          -0.3468, -0.1601,  0.0986]],

        [[ 0.2981, -0.0916, -0.2575,  0.1779,  0.1415,  0.2658,  0.1996,
          -0.0085, -0.0554,  0.0943, -0.3817, -0.1044,  0.1272, -0.1162,
          -0.1962,  0.1500,  0.0625, -0.0933, -0.2830,  0.1111, -0.1068,
        

# SwiGLU

## 常见的激活函数
* ReLU

    $ReLU(x) = \max(0, x)$

* GeLU

    论文《Gaussian Error Linear Units（GELUs）》提出了GELU，GeLU激活函数是处处可微的非线性函数，比ReLU更平滑

* Swish

    论文《Swish: a Self-Gated Activation Function》提出了Swish，这也是对带有非零负值梯度的ReLU平滑版本，同样是个处处可微的非线性函数，且有一个参数beta用于控制函数的形状。

* Silu

    beta=1的swish函数

![comparation](pic/af.png)

## GLU

GLU（Gated Linear Units）其实不算是一种激活函数，而是一种神经网络层。它是一个线性变换后面接门控机制的结构。其中门控机制是一个sigmoid函数用来控制信息能够通过多少。

$GLU(x,W,V,b,c) = \sigma (xW + b) \odot (xV + c)$

这里$\sigma$代表SIGMOD函数，$\odot$代表逐元素乘。

## SwiGLU

把上面公式的激活函数换成swish函数：

$GLU(x,W,V,b,c) = Swish_1 (xW + b) \odot (xV + c)$

就是SwiGLU



In [9]:
class SwiGLU(nn.Module):
    def __init__(self, input_dim,  multiple_of=64) -> None:
        super().__init__()
        self.input_dim = input_dim

        self.hidden_dim = 4 * self.input_dim
        self.hidden_dim = int(2 * self.hidden_dim / 3)
        self.hidden_dim = multiple_of * ((self.hidden_dim + multiple_of - 1) // multiple_of)

        self.w1 = nn.Linear(self.input_dim, self.hidden_dim)
        self.w2 = nn.Linear(self.hidden_dim, self.input_dim)
        self.w3 = nn.Linear(self.input_dim, self.hidden_dim)
        self.silu = nn.SiLU()
    
    def forward(self, x):
        return self.w2(self.silu(self.w1(x)) * self.w3(x))



## 代码测试

In [10]:
x = torch.rand(2, 3, 24)
mlp = SwiGLU(24)
x = mlp(x)
print("x.shape:", x.shape)
x

x.shape: torch.Size([2, 3, 24])


tensor([[[ 0.0863,  0.0724, -0.1174, -0.0178,  0.1053, -0.0813, -0.0694,
          -0.0511, -0.0421, -0.0691,  0.0864,  0.1182, -0.0145, -0.0499,
          -0.1224, -0.0376, -0.0461,  0.0983,  0.1722,  0.0814, -0.0801,
           0.0901,  0.1629,  0.1242],
         [ 0.0915,  0.0239, -0.0666,  0.0105,  0.0647, -0.0958, -0.0596,
          -0.0934, -0.0255, -0.0773,  0.0648,  0.1017, -0.0262, -0.0217,
          -0.1037,  0.0142, -0.0580,  0.0730,  0.1273,  0.1023, -0.0219,
           0.1028,  0.0771,  0.0926],
         [ 0.1344,  0.0964, -0.1336, -0.0356,  0.1447, -0.1304, -0.0404,
          -0.0523, -0.0413, -0.1157,  0.0330,  0.1496, -0.0723, -0.0082,
          -0.1495,  0.0415, -0.0442,  0.1508,  0.2035,  0.0816, -0.0606,
           0.1507,  0.1280,  0.1085]],

        [[ 0.1096,  0.0675, -0.0817,  0.0050,  0.0828, -0.0968, -0.0621,
          -0.0968, -0.0390, -0.0669,  0.0421,  0.1272, -0.0210, -0.0211,
          -0.0842, -0.0090, -0.0642,  0.0838,  0.1421,  0.0876, -0.0172,
        

## 值得一提的是

### 为什么要重新计算隐藏层维度

传统FFN使用4倍的input_dim作为hidden_dim（见上述MLP代码），此时不妨do the math：

* 传统FFN使用了两个线性层，参数量为 $ dim \times 4dim \times 2$
* SwiGLU使用了三个线性层，参数量为 $ dim \times hidden \_ dim  \times 3$
* 想要保证传统FFN与SwiGLU参数量上不发生太大变化，那么 $ hidden \_ dim $选择为 $ \cfrac{2}{3} \times 4dim $ 是最好的
* 然而NVIDIA GPU 的 Tensor Core 要求矩阵维度是 8/16/32/256 的倍数才能发挥最大算力，因此还得将$ hidden \_ dim $取值为8/16/32/256 的倍数，同时接近$ \cfrac{2}{3} \times 4dim $

### 取整技巧

其中

        self.hidden_dim = multiple_of * ((self.hidden_dim + multiple_of - 1) // multiple_of)

这个操作是将self.hidden_dim重新取值为可以被multiple_of整除的数，数学表达为：

$ceil(x / m) = (x + m - 1) // m$

* **情况1**: x 恰好是 m 的倍数，设 x = k*m

    (k*m + m - 1) // m = k + (m-1)//m = k + 0 = k  ✓

* **情况2**: x 有余数，设 x = k*m + r (0 < r < m)

    (k*m + r + m - 1) // m = k + (r + m - 1)//m = k + 1  ✓


# MoE


## 何为MoE

传统FFN层是稠密的，参考下面的图片：

![FFN](pic/FFN.jpg)

我们可以把一个密集模型拆成若干片，然后重新训练。在模型实际运行时，每次只激活其中的一小部分：

![FFN](pic/MoE.jpg)

## 为何用MoE

### 一些有趣的事实(摘自知乎用户@阿哈)

根据 Meta 发布的《The Llama 3 Herd of Models》技术报告，在训练 Llama 3 405B （最大的那个 Dense版本）的过程中，训练任务总共经历了 466 次 中断并重启。

Meta的Llama 3-405B模型训练过程非常复杂。该项目在54天的预训练中，使用了超过16,000张H100显卡，共经历了466次中断，平均每三小时发生一次。其中绝大多数（419次）是意外中断，主要由GPU硬件故障（如NVLink或HBM3问题）引起。尽管中断频繁，但Meta通过自动化系统将有效训练时间维持在90%以上，证明了其在管理超大规模硬件集群方面的工程能力。

与Llama 3的硬件挑战形成鲜明对比，DeepSeek-V3的训练过程则展示了算法架构优化的巨大成功。尽管DeepSeek-V3是一个参数量达6710亿的MoE（混合专家）模型，且训练了14.8T Token，但其官方报告称整个预训练过程实现了“零次回滚”，没有发生任何不可恢复的训练发散。这一成就主要归功于其创新的“无辅助损失”负载均衡策略和稳定的FP8混合精度训练，这些技术从根源上解决了MoE模型难以训练的难题，证明了先进的算法设计甚至可以超越硬件规模带来的不稳定性，用moe模型更加可以将模型堆得很大。

![vs](pic/MoE_vs_Dens.png)

## MoE的训练

以下摘自[huggingface](https://huggingface.co/blog/zh/moe)

![vs](pic/EP.png)

GShard 将在编码器和解码器中的每个前馈网络 (FFN) 层中的替换为使用 Top-2 门控的混合专家模型 (MoE) 层。下图展示了编码器部分的结构。这种架构对于大规模计算非常有效: 当扩展到多个设备时，MoE 层在不同设备间共享，而其他所有层则在每个设备上复制。

这意味着一张卡上可以只放一个专家对应的shard，除了专家权重以外其他权重都共享。模型训练时如果门控激活了某几个专家，那么这几张卡就进行前向计算。

## 专家们学到了啥

ST-MoE 的研究者们发现，编码器中不同的专家倾向于专注于特定类型的令牌或浅层概念。例如，某些专家可能专门处理标点符号，而其他专家则专注于专有名词等。与此相反，解码器中的专家通常具有较低的专业化程度。此外，研究者们还对这一模型进行了多语言训练。尽管人们可能会预期每个专家处理一种特定语言，但实际上并非如此。由于令牌路由和负载均衡的机制，没有任何专家被特定配置以专门处理某一特定语言。

就是说解码器中的专家们更倾向于关注句法结构。

![E](pic/experts.png)

## 专家的数量对预训练的影响

增加更多专家可以提升处理样本的效率和加速模型的运算速度，但这些优势随着专家数量的增加而递减 (尤其是当专家数量达到 256 或 512 之后更为明显)。

就是说专家不是越多越好。

同时，这也意味着在推理过程中，需要更多的显存来加载整个模型。值得注意的是，Switch Transformers 的研究表明，其在大规模模型中的特性在小规模模型下也同样适用，即便是每层仅包含 2、4 或 8 个专家。

## MoE模型的微调

稠密模型和稀疏模型在过拟合的动态表现上存在显著差异。稀疏模型更易于出现过拟合现象，因此在处理这些模型时，尝试更强的内部正则化措施是有益的，比如使用更高比例的 dropout。例如，我们可以为稠密层设定一个较低的 dropout 率，而为稀疏层设置一个更高的 dropout 率，以此来优化模型性能。

在微调过程中是否使用辅助损失是一个需要决策的问题。ST-MoE 的作者尝试关闭辅助损失，发现即使高达 11% 的令牌被丢弃，模型的质量也没有显著受到影响。令牌丢弃可能是一种正则化形式，有助于防止过拟合。

Switch Transformers 的作者观察到，在相同的预训练困惑度下，稀疏模型在下游任务中的表现不如对应的稠密模型，特别是在重理解任务 (如 SuperGLUE) 上。另一方面，对于知识密集型任务 (如 TriviaQA)，稀疏模型的表现异常出色。作者还观察到，在微调过程中，较少的专家的数量有助于改善性能。另一个关于泛化问题确认的发现是，模型在小型任务上表现较差，但在大型任务上表现良好。

![T](pic/MoE_task.png)

这里图中的结论是小任务中MoE表现差，大任务中MoE表现好。

一种可行的微调策略是尝试冻结所有非专家层的权重。实践中，这会导致性能大幅下降，但这符合我们的预期，因为混合专家模型 (MoE) 层占据了网络的主要部分。我们可以尝试相反的方法: 仅冻结 MoE 层的参数。实验结果显示，这种方法几乎与更新所有参数的效果相当。这种做法可以加速微调过程，并降低显存需求。

![F](pic/Frozen.png)

图中说明了冻结MoE层的参数时，模型在测试任务上的跑分跟不冻结任何参数没太大差距。那么可以选择仅仅冻结MoE层，这样会在保持质量的同时加快训练速度。

在微调稀疏混合专家模型 (MoE) 时需要考虑的最后一个问题是，它们有特别的微调超参数设置——例如，稀疏模型往往更适合使用较小的批量大小和较高的学习率，这样可以获得更好的训练效果。

![F](pic/lr.png)

In [11]:
class SMoE(nn.Module):
    def __init__(self, input_dim, expert_num, top_k = 4) -> None:
        super().__init__()
        self.top_k = top_k
        self.input_dim = input_dim
        self.expert_num = expert_num
        self.ffn = SwiGLU(input_dim) # 这里的FFN选择用了前面的SwiGLU
        self.experts = nn.ModuleList([self.ffn for _ in range(self.expert_num)])
        self.router = nn.Linear(self.input_dim, self.expert_num)
    def forward(self, x):
        bs, seq, _ = x.shape
        x = x.view(-1, self.input_dim)
        router_logits = self.router(x)
        router_scores = torch.softmax(router_logits, dim=-1)
        topk_probs, topk_indices = torch.topk(router_scores, self.top_k, dim=-1, sorted=False)# N self.top_k
        topk_probs = topk_probs / (topk_probs.sum(dim=-1, keepdim=True) + 1e-20)
        x_r = x.repeat_interleave(self.top_k, dim=0)# N*self.top_k D
        out = torch.empty_like(x_r)
        topk_indices_flat = topk_indices.view(-1)# N*self.top_k
        for expert_id, expert in enumerate(self.experts):
            out[expert_id == topk_indices_flat] = expert(x_r[expert_id == topk_indices_flat])
        
        out = (out.view(*topk_probs.shape, -1) * topk_probs.unsqueeze(-1)).sum(1)# N D

        out = out.view(bs, seq, -1)

        return out

## 代码测试

In [12]:

x = torch.rand(2, 4, 64)

moe = SMoE(64, 8)

out = moe(x)

out.shape

torch.Size([2, 4, 64])